# IMPORTING LIBRARIES

In [2]:
file_location_images = '/Users/vishaljha/Documents/GitHub/ML Playground/ImageCaptioning/Model/Dataset/Flickr_Data/Flickr_Data/Images/'
len(file_location_images)

109

In [137]:
import os
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"

os.environ["RUNFILES_DIR"] = "/Library/Frameworks/Python.framework/Versions/3.7/share/plaidml"
# plaidml might exist in different location. Look for "/usr/local/share/plaidml" and replace in above path

os.environ["PLAIDML_NATIVE_PATH"] = "/Library/Frameworks/Python.framework/Versions/3.7/lib/libplaidml.dylib"
# libplaidml.dylib might exist in different location. Look for "/usr/local/lib/libplaidml.dylib" and replace in above path

In [139]:
import glob
from tqdm import tqdm

import pandas as pd
import numpy as np

import pickle

from tensorflow.keras.preprocessing import image, sequence
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import LSTM, Embedding, TimeDistributed, Dense, RepeatVector, concatenate, Activation, Flatten, Concatenate
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.layers import Bidirectional


In [32]:
file_location_caption = '/Users/vishaljha/Documents/GitHub/ML Playground/ImageCaptioning/Model/Dataset/Flickr_Data/Flickr_Data/Flickr_TextData/Flickr8k.token.txt'
file_location_images = '/Users/vishaljha/Documents/GitHub/ML Playground/ImageCaptioning/Model/Dataset/Flickr_Data/Flickr_Data/Images/'

file_location_train_images = '/Users/vishaljha/Documents/GitHub/ML Playground/ImageCaptioning/Model/Dataset/Flickr_Data/Flickr_Data/Flickr_TextData/Flickr_8k.trainImages.txt'
file_location_val_images = '/Users/vishaljha/Documents/GitHub/ML Playground/ImageCaptioning/Model/Dataset/Flickr_Data/Flickr_Data/Flickr_TextData/Flickr_8k.devImages.txt'
file_location_test_images = '/Users/vishaljha/Documents/GitHub/ML Playground/ImageCaptioning/Model/Dataset/Flickr_Data/Flickr_Data/Flickr_TextData/Flickr_8k.testImages.txt'

file_location_training_data = '/Users/vishaljha/Documents/GitHub/ML Playground/ImageCaptioning/Model/Dataset/Flickr_Data/Flickr_Data/flickr8ktextfiles/flickr_8k_train_dataset.txt'

# GENERATING A DICTIONARY OF CAPTIONS

In [4]:
captions = open(file_location_caption, 'r').read().strip().split('\n')

In [5]:
dictionary_captions = {}
for i, row in enumerate(captions):
    row = row.split('\t')
    row[0] = row[0][:len(row[0])-2]
    if row[0] in dictionary_captions:
        dictionary_captions[row[0]].append(row[1])
    else:
        dictionary_captions[row[0]] = [row[1]]

In [6]:
dictionary_captions['3637013_c675de7705.jpg']

["A couple stands close at the water 's edge .",
 'The two people stand by a body of water and in front of bushes in fall .',
 'Two people hold each other near a pond .',
 'Two people stand by the water .',
 'Two people stand together on the edge of the water on the grass .']

# GETTING ALL TRAINING, VALIDATION, TESTING IMAGES

In [7]:
global img 
img = glob.glob(file_location_images+'*.jpg')
img[:5]

['/Users/vishaljha/Documents/GitHub/ML Playground/ImageCaptioning/Model/Dataset/Flickr_Data/Flickr_Data/Images/2387197355_237f6f41ee.jpg',
 '/Users/vishaljha/Documents/GitHub/ML Playground/ImageCaptioning/Model/Dataset/Flickr_Data/Flickr_Data/Images/2609847254_0ec40c1cce.jpg',
 '/Users/vishaljha/Documents/GitHub/ML Playground/ImageCaptioning/Model/Dataset/Flickr_Data/Flickr_Data/Images/2046222127_a6f300e202.jpg',
 '/Users/vishaljha/Documents/GitHub/ML Playground/ImageCaptioning/Model/Dataset/Flickr_Data/Flickr_Data/Images/2853743795_e90ebc669d.jpg',
 '/Users/vishaljha/Documents/GitHub/ML Playground/ImageCaptioning/Model/Dataset/Flickr_Data/Flickr_Data/Images/2696951725_e0ae54f6da.jpg']

In [8]:
images_training = set(open(file_location_train_images, 'r').read().strip().split('\n'))
images_validation = set(open(file_location_val_images, 'r').read().strip().split('\n')) 
images_testing = set(open(file_location_test_images, 'r').read().strip().split('\n'))

In [9]:
def split_image_data(l):
    temp = []
    for i in img:
        if i[len(file_location_images):] in l:
            temp.append(i)
    return temp

In [10]:
train = split_image_data(images_training)
val = split_image_data(images_validation)
test = split_image_data(images_testing)

In [11]:
len(train), len(val), len(test)

(6000, 1000, 1000)

# PREPROCESSING IMAGES

In [12]:
def preprocess_input(x):
    x /= 255.
    x -= 0.5
    x += 2.
    return x

In [13]:
def preprocess_image_data(image_path):
    img = image.load_img(image_path, target_size=(299, 299))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)

    x = preprocess_input(x)
    return x

# LOAD VGG-16 MODEL TRAINED ON IMAGE-NET DATASET

In [14]:
model = InceptionV3(weights='imagenet')

new_input = model.input
hidden_layer = model.layers[-2].output

model_new = Model(new_input, hidden_layer)

# ENCODING ALL DATASET USING ALTERED VGG-16

In [15]:
def encode_image(image):
    image = preprocess_image_data(image)
    temp = model_new.predict(image)
    temp = np.reshape(temp, temp.shape[1])
    return temp

In [16]:
train_data_encode = {}
for imag in tqdm(train):
    train_data_encode[imag[len(file_location_images):]] = encode_image(imag)



100%|██████████| 6000/6000 [10:22<00:00,  9.64it/s]


In [17]:
with open("/Users/vishaljha/Documents/GitHub/ML Playground/ImageCaptioning/Model/pickel_store/encoded_images_train.p", "wb") as encoded_pickle:
    pickle.dump(train_data_encode, encoded_pickle)

In [18]:
test_data_encode = {}
for img in tqdm(test):
    test_data_encode[img[len(file_location_images):]] = encode_image(img)

100%|██████████| 1000/1000 [02:04<00:00,  8.02it/s]


In [19]:
with open("/Users/vishaljha/Documents/GitHub/ML Playground/ImageCaptioning/Model/pickel_store/encoded_images_test.p", "wb") as encoded_pickle:
    pickle.dump(test_data_encode, encoded_pickle)

In [20]:
train_data_encode[train[0][len(file_location_images):]].shape, test_data_encode[test[0][len(file_location_images):]].shape

((2048,), (2048,))

In [21]:
len(test_data_encode), len(train_data_encode)

(1000, 6000)

In [22]:
train_d = {}
for i in train:
    if i[len(file_location_images):] in dictionary_captions:
        train_d[i] = dictionary_captions[i[len(file_location_images):]]

val_d = {}
for j in val:
    if j[len(file_location_images):] in dictionary_captions:
        val_d[j] = dictionary_captions[j[len(file_location_images):]]

test_d = {}
for k in test:
    if k[len(file_location_images):] in dictionary_captions:
        test_d[k] = dictionary_captions[k[len(file_location_images):]]

In [23]:
len(train_d), len(val_d), len(test_d)

(6000, 1000, 1000)

# CALCULATING UNIQUE WORDS IN CAPTIONS, MAPPING UNIQUE WORDS TO INDICES AND VICE-VERSA IN 2 SEPERATE DICTIONARIES, MAX_LENGTH OF CAPTIONS

In [24]:
caps = []
for key, value in train_d.items():
    for i in value:
        caps.append('<start> ' + i + ' <end>')

cap_words = [i.split() for i in caps]

unique_captions = []
for i in cap_words:
    unique_captions.extend(i)

unique_captions = list(set(unique_captions))

In [25]:
with open("/Users/vishaljha/Documents/GitHub/ML Playground/ImageCaptioning/Model/pickel_store/unique_captions.p", "wb") as i:
    pickle.dump(unique_captions, i)

In [26]:
unique = pickle.load(open("/Users/vishaljha/Documents/GitHub/ML Playground/ImageCaptioning/Model/pickel_store/unique_captions.p", "rb"))
len(unique)

8256

In [27]:
word2idx = {val:index for index, val in enumerate(unique)}
idx2word = {index:val for index, val in enumerate(unique)}

In [28]:
word2idx['<start>'], idx2word[3482]

(4776, 'classes')

In [29]:
max_cap_length = 0
for c in caps:
    c = c.split()
    if len(c) > max_cap_length:
        max_cap_length = len(c)

vocab_size = len(unique)

In [31]:
max_cap_length, vocab_size

(40, 8256)

# ADDING < START> AND < END> TAGS TO INDICATE STARTING AND ENDING OF A SENTENCE

In [150]:
f = open(file_location_training_data, 'w')
f.write("image_id\tcaptions\n")

18

In [151]:
for key, value in train_d.items():
    for i in value:
        f.write(key[len(file_location_images):] + '\t' + '<start> ' + i + ' <end>' + '\n')
f.close()

In [152]:
df = pd.read_csv(file_location_training_data, delimiter='\t')
c = [i for i in df['captions']]

len(df), len(c)

(30000, 30000)

In [153]:
imgs = [i for i in df['image_id']]

a = c[-1]
a, imgs[-1]

('<start> A longeared dog running in the grass . <end>',
 '2764732789_1392e962d0.jpg')

In [154]:
sample_per_epochs = 0
for ca in caps:
    sample_per_epochs += len(ca.split()) - 1

sample_per_epochs

383454

# CAPTION GENERATOR

In [155]:
def data_generator(batch_size= 64):
    partial_captions = []
    next_word = []
    images= []

    df = pd.read_csv(file_location_training_data, delimiter='\t')
    df = df.sample(frac=1)
    iter = df.iterrows()

    c = []
    imgs = []

    for i in range(df.shape[0]):
        x = next(iter)
        c.append(x[1][1])
        imgs.append(x[1][0])

    count = 0
    while True:
        for idx, text in enumerate(c):
            current_image = train_data_encode[imgs[idx]]

            for i in range(len(text.split()) - 1):
                count += 1

                partial = [word2idx[txt] for txt in text.split()[:i+1]]
                partial_captions.append(partial)

                n = np.zeros(vocab_size)
                n[word2idx[text.split()[i+1]]] = 1
                next_word.append(n)

                images.append(current_image)

                if count >= batch_size:
                    next_word = np.asarray(next_word)
                    images = np.asarray(images)
                    partial_captions = sequence.pad_sequences(partial_captions, maxlen=max_cap_length, padding='post')
                    yield[[images, partial_captions], next_word]

                    partial_captions = []
                    next_word = []
                    images = []
                    count = 0